In [2]:
import os
import numpy as np
import scanpy as sc
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

In [3]:
data_path = "data/"
check_path = "checkpoints/"
table_path = "tables/"

# os.makedirs(check_path)

os.chdir("../")
print(os.getcwd())

/Users/flynnzhang/CMU/Spring24/02620-ML4Scientists/scRNA-seq_ML


In [3]:
# load all combined data
all_adata = sc.read_h5ad(check_path + 'combined_labeled.h5ad')

all_adata

AnnData object with n_obs × n_vars = 23873 × 19724
    obs: 'Sample', 'Title', 'Marker', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'pct_counts_mt', 'pct_counts_ribo', 'pct_counts_hb', 'batch', 'kmeans_clusters', 'cell_types'
    var: 'n_cells'
    uns: 'log1p'
    layers: 'counts'

In [4]:
all_adata.obs.cell_types.value_counts()

cell_types
immune cells          10372
fibroblasts            9046
epithelial cells       3181
endothelial cells      1039
smooth muscle cell      235
Name: count, dtype: int64

In [5]:
# subset data to only include cells from cell types of interest
cell_types_of_interest = ['epithelial cells']
adata_subset = all_adata[all_adata.obs.cell_types.isin(cell_types_of_interest)].copy()

adata_subset

AnnData object with n_obs × n_vars = 3181 × 19724
    obs: 'Sample', 'Title', 'Marker', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'pct_counts_mt', 'pct_counts_ribo', 'pct_counts_hb', 'batch', 'kmeans_clusters', 'cell_types'
    var: 'n_cells'
    uns: 'log1p'
    layers: 'counts'

In [6]:
# load all genes related to TP53, first row is induced, second is repressed
tp53_genes = pd.read_csv(data_path + 'TP53/genes_related.txt', sep='\t')
# Uppercase only the first letter
tp53_genes['Gene'] = tp53_genes['Gene'].str.capitalize()
induced = tp53_genes[tp53_genes['Relation']=='induced']['Gene']
repressed = tp53_genes[tp53_genes['Relation']=='repressed']['Gene']

len(induced), len(repressed)

(20, 10)

In [7]:
# See if TP53-related genes are present in the data
induced_present = [gene for gene in induced if gene in adata_subset.var_names]
repressed_present = [gene for gene in repressed if gene in adata_subset.var_names]

print(len(induced_present))
print(len(repressed_present))

18
10


# Regression

#### List of cell types that are worth exploring:
- Immune cells
- Fibroblasts
- Endothelial cells
- Epithelial cells

TFs downloaded form [tfcheckpoint](https://www.tfcheckpoint.org/)

In [8]:
# load all tfs
tfs = pd.read_csv(data_path + "TFs/tfs.txt", sep="\t")

# we need mouse tfs
TFs_mouse = tfs[tfs['Mouse_entrez'].notna()]['Gene_symbol'].values
print('Total TFs of mouse:', len(TFs_mouse))
TFs_mouse = [x for x in TFs_mouse if x in adata_subset.var_names]

print('Total genes in', cell_types_of_interest,len(adata_subset.var_names))
print('Total TFs present in', cell_types_of_interest, len(TFs_mouse))

Total TFs of mouse: 2733
Total genes in ['epithelial cells'] 19724
Total TFs present in ['epithelial cells'] 57


In [9]:
# subset data to only include TP53-related genes and present TFs
# use unique
genes_of_interest = np.unique(list(induced_present) + list(repressed_present) + list(TFs_mouse))
adata_sub_hp53 = adata_subset[:, genes_of_interest].copy()

adata_sub_hp53

AnnData object with n_obs × n_vars = 3181 × 85
    obs: 'Sample', 'Title', 'Marker', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'pct_counts_mt', 'pct_counts_ribo', 'pct_counts_hb', 'batch', 'kmeans_clusters', 'cell_types'
    var: 'n_cells'
    uns: 'log1p'
    layers: 'counts'

In [10]:
# label genes with TF, target, or both in adata
adata_sub_hp53.var['gene_type'] = 'target'
adata_sub_hp53.var.loc[adata_sub_hp53.var_names.isin(TFs_mouse), 'gene_type'] = 'TF'


# Those that are both in TFs_mouse and induced_present or repressed_present
adata_sub_hp53.var.loc[adata_sub_hp53.var_names.isin(tp53_genes['Gene'].to_list()) & adata_sub_hp53.var_names.isin(TFs_mouse), 'gene_type'] = 'both'

adata_sub_hp53.var

,n_cells,gene_type
A430033K04Rik,199,TF
Acad11,1573,target
Aen,6031,target
Aicda,23,TF
Aldh4a1,1201,target
...,...,...
Zfp809,1528,TF
Zfp84,3233,TF
Zfp955a,726,TF
Zim1,43,TF


In [11]:
adata_sub_hp53

AnnData object with n_obs × n_vars = 3181 × 85
    obs: 'Sample', 'Title', 'Marker', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'pct_counts_mt', 'pct_counts_ribo', 'pct_counts_hb', 'batch', 'kmeans_clusters', 'cell_types'
    var: 'n_cells', 'gene_type'
    uns: 'log1p'
    layers: 'counts'

In [12]:
# remove genes with 0 variance
variances = np.var(adata_sub_hp53.X.toarray(), axis=0)
non_zero_variance_mask = variances > 0
adata_sub_hp53 = adata_sub_hp53[:, non_zero_variance_mask]

adata_sub_hp53

View of AnnData object with n_obs × n_vars = 3181 × 81
    obs: 'Sample', 'Title', 'Marker', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'pct_counts_mt', 'pct_counts_ribo', 'pct_counts_hb', 'batch', 'kmeans_clusters', 'cell_types'
    var: 'n_cells', 'gene_type'
    uns: 'log1p'
    layers: 'counts'

In [13]:
# use combat to correct for batch effects and rule out unwanted sources of variation
sc.pp.combat(adata_sub_hp53, key='batch', covariates=['total_counts'])

# scale each gene to unit variance, clip values exceeding SD 10
sc.pp.scale(adata_sub_hp53, max_value=10)

/Users/flynnzhang/anaconda3/envs/seq/lib/python3.12/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  df[key] = c


In [15]:
adata_sub_hp53

AnnData object with n_obs × n_vars = 3181 × 81
    obs: 'Sample', 'Title', 'Marker', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'pct_counts_mt', 'pct_counts_ribo', 'pct_counts_hb', 'batch', 'kmeans_clusters', 'cell_types'
    var: 'n_cells', 'gene_type', 'mean', 'std'
    uns: 'log1p'
    layers: 'counts'

In [14]:
# write to h5ad
adata_sub_hp53.write_h5ad(check_path + 'ready_for_regression.h5ad')

## OLS and Ridge Regression

In [4]:
# load adata that's ready for regression
adata_sub_hp53 = sc.read_h5ad(check_path + 'ready_for_regression.h5ad')

adata_sub_hp53

AnnData object with n_obs × n_vars = 3181 × 81
    obs: 'Sample', 'Title', 'Marker', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'pct_counts_mt', 'pct_counts_ribo', 'pct_counts_hb', 'batch', 'kmeans_clusters', 'cell_types'
    var: 'n_cells', 'gene_type', 'mean', 'std'
    uns: 'log1p'
    layers: 'counts'

In [5]:
import statsmodels.api as sm
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

# OLS and Ridge regression
# alpha: regularization strength for Ridge regression
def fit_model(adata, method, alpha=1):
    results = {}
    tf_list = adata.var[(adata.var['gene_type'] == 'TF') | (adata.var['gene_type'] == 'both')].index
    
    print('Initializing ' + method + ' regression model...')
    
    for tf in tf_list:
        print('Fitting models for', tf)
        Y = adata[:, tf].X
        
        # X are expr of all target genes or both
        genes_for_X = adata.var[(adata.var['gene_type'] != 'TF') & (adata.var['gene_type'] != tf)].index
        X = adata.X[:, adata.var_names.isin(genes_for_X)]
        
        # Standardize features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # OLS regression
        if method == 'OLS':
            X_ols = sm.add_constant(X_scaled)
            model = sm.OLS(Y, X_ols)
            result = model.fit()
            

        # Ridge regression
        if method == 'Ridge':
            model = Ridge(alpha=alpha)
            result = model.fit(X_scaled, Y)
            
        # Save the results
        results[tf] = result
        
    return results

In [6]:
# Fit the models
ols_res = fit_model(adata_sub_hp53, 'OLS')

Initializing OLS regression model...
Fitting models for A430033K04Rik


Fitting models for Arap1
Fitting models for Arglu1
Fitting models for Arhgap22
Fitting models for Asf1a
Fitting models for Basp1
Fitting models for Borcs8
Fitting models for Btn1a1
Fitting models for Cdc42bpg
Fitting models for Chmp1a
Fitting models for Csprs
Fitting models for Cys1
Fitting models for Dpy30
Fitting models for Ecsit
Fitting models for Eif4a3
Fitting models for Elp4
Fitting models for Fgf11
Fitting models for Fgf2
Fitting models for Gnas
Fitting models for Ikbkg
Fitting models for Med29
Fitting models for Msx3
Fitting models for Ndufa13
Fitting models for Npm2
Fitting models for Nup62
Fitting models for P2rx2
Fitting models for Pak6
Fitting models for Ppp1r15a
Fitting models for Pus1
Fitting models for Rad54l2
Fitting models for Rhox5
Fitting models for Ripply1
Fitting models for Rnf141
Fitting models for Rpf2
Fitting models for Rpl11
Fitting models for Setdb2
Fitting models for Spop
Fitting models for Traf7
Fitting models for Treml1
Fitting models for Trib2
Fitting mode

In [7]:
# get r-sqaure for all models
r2_ols = {tf: res.rsquared for tf, res in ols_res.items()}
r2_ols = pd.Series(r2_ols)

r2_ols

A430033K04Rik    0.005493
Arap1            0.011142
Arglu1           0.030694
Arhgap22         0.017159
Asf1a            0.037474
Basp1            0.070636
Borcs8           0.025874
Btn1a1           0.010331
Cdc42bpg         0.010972
Chmp1a           0.022253
Csprs            0.054305
Cys1             0.017219
Dpy30            0.091057
Ecsit            0.032886
Eif4a3           0.061292
Elp4             0.013853
Fgf11            0.009001
Fgf2             0.015072
Gnas             0.052085
Ikbkg            0.015430
Med29            0.042068
Msx3             0.005800
Ndufa13          0.068734
Npm2             0.012148
Nup62            0.050585
P2rx2            0.092757
Pak6             0.031428
Ppp1r15a         0.305253
Pus1             0.041181
Rad54l2          0.020950
Rhox5            0.029514
Ripply1          0.002845
Rnf141           0.012362
Rpf2             0.044848
Rpl11            0.151423
Setdb2           0.010976
Spop             0.020269
Traf7            0.029342
Treml1      